In [2]:
## data Ingestion
#load
from langchain_community.document_loaders import TextLoader
import os
text=TextLoader("Malaria.txt",  encoding="utf-8")
text_documents=text.load()
text_documents

[Document(metadata={'source': 'Malaria.txt'}, page_content="Malaria: An Overview\n\nMalaria is a life-threatening disease caused by parasites that are transmitted to people through the bites of infected female Anopheles mosquitoes. It remains one of the most significant public health challenges in the world, particularly in tropical and subtropical regions. According to the World Health Organization (WHO), in 2022, there were an estimated 249 million malaria cases and over 600,000 malaria-related deaths globally, with the highest burden in sub-Saharan Africa.\n\nCauses and Transmission\nMalaria is caused by Plasmodium parasites. There are five species of Plasmodium that cause malaria in humans:\n\nPlasmodium falciparum (most deadly and common in Africa)\n\nPlasmodium vivax (common in Asia and Latin America)\n\nPlasmodium malariae\n\nPlasmodium ovale\n\nPlasmodium knowlesi (zoonotic, found in Southeast Asia)\n\nThe parasites are transmitted via the bites of infected Anopheles mosquitoes

In [2]:
## web base loader
from langchain_community.document_loaders import WebBaseLoader
import bs4

textloader = WebBaseLoader(
    web_path="https://python.langchain.com/docs/introduction/",
    bs_kwargs=dict(parse_only=bs4.SoupStrainer("body"))
)

text=textloader.load()
text

USER_AGENT environment variable not set, consider setting it to identify your requests.


[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/'}, page_content='\nSkip to main contentWe are growing and hiring for multiple roles for LangChain, LangGraph and LangSmith.  Join our team!IntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memory to chatbotsHow to use example selectorsHow to add a semantic layer over graph databaseHow to invoke runnables in parall

In [3]:
#transform
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = text_splitter.split_documents(text_documents)
# print(len(chunks))
chunks[4]

Document(metadata={'source': 'Malaria.txt'}, page_content='Fever and chills\n\nSweating\n\nHeadache\n\nMuscle and joint pain\n\nFatigue\n\nNausea and vomiting\n\nAnemia and jaundice due to destruction of red blood cells\n\nIf not treated promptly and correctly, P. falciparum malaria can progress to severe illness, often leading to death. Severe symptoms may include:\n\nCerebral malaria (coma or seizures)\n\nRespiratory distress\n\nOrgan failure (kidneys, liver, etc.)\n\nHypoglycemia\n\nDiagnosis\nPrompt diagnosis is critical. Malaria is typically diagnosed using:')

In [5]:
## vector Embedding
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
db= FAISS.from_documents(chunks, HuggingFaceEmbeddings())

C:\Users\pthorat\AppData\Local\Temp\ipykernel_9996\2433880375.py:4: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  db= FAISS.from_documents(chunks, HuggingFaceEmbeddings())


In [ ]:
query= "precautions for malaria?"
result=db.similarity_search(query)
combined_result = " ".join([doc.page_content for doc in result])
len(result)
# combined_result

4

In [44]:
result[0].page_content

'Fever and chills\n\nSweating\n\nHeadache\n\nMuscle and joint pain\n\nFatigue\n\nNausea and vomiting\n\nAnemia and jaundice due to destruction of red blood cells\n\nIf not treated promptly and correctly, P. falciparum malaria can progress to severe illness, often leading to death. Severe symptoms may include:\n\nCerebral malaria (coma or seizures)\n\nRespiratory distress\n\nOrgan failure (kidneys, liver, etc.)\n\nHypoglycemia\n\nDiagnosis\nPrompt diagnosis is critical. Malaria is typically diagnosed using:'

In [6]:
# from langchain_community.llms import Ollama
# llm= Ollama(model="llama2")
from langchain_community.chat_models import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()
llm= ChatOpenAI(model='gpt-4o-mini', max_tokens=200)

C:\Users\pthorat\AppData\Local\Temp\ipykernel_9996\3834886298.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm= ChatOpenAI(model='gpt-4o-mini', max_tokens=200)


In [7]:
from langchain_core.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate.from_template(
    """Answer the following questions based on the provided contex.
    Think step by step before providing a detailed answer.
    <context>
    {context}
    </context>
    Question:{input}"""
)

In [8]:
## Building chain
from langchain.chains import RetrievalQA
retriever= db.as_retriever()
# print(retriever)
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever)

In [9]:
chain=prompt|llm

In [ ]:
input_text = input("Ask your question: ")
# relevant_content=db.similarity_search(input_text, k=5)[:4].page_content
relevant_content=db.similarity_search(input_text)
combined_content="".join([doc.page_content for doc in relevant_content])
response = chain.invoke({
    "context": combined_content,
    "input": input_text
})
print(response)


content='Malaria is a life-threatening disease caused by parasites. These parasites are transmitted to humans through the bites of infected female Anopheles mosquitoes. The disease poses a significant public health challenge, particularly in tropical and subtropical regions of the world. \n\nIn terms of its impact, malaria continues to be a major concern, as evidenced by the World Health Organization (WHO) data from 2022, which reported approximately 249 million cases and over 600,000 deaths related to malaria globally. The highest burden of the disease is found in sub-Saharan Africa, highlighting the critical need for effective prevention and treatment strategies in these areas.' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 126, 'prompt_tokens': 137, 'total_tokens': 263, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, '